In [1]:
pip install accelerate sentencepiece gradio tensorboard tensorboardX optuna datasets peft bitsandbytes wandb torch transformers evaluate rouge_score nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 113.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 167.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 122.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 138.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 212.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 119.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 195.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 117.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 184.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 169.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 181.2 MB/s eta 0:00:00
  Attempting uninstall: dill
    

In [2]:
from huggingface_hub import login

login()

In [3]:
import ipywidgets as widgets
from IPython.display import display

# Define available models
model_options = ["Llama 3", "Mistral", "SmolLM"]

# Create a dropdown widget
model_dropdown = widgets.Dropdown(
    options=model_options,
    description="Model:",
    style={'description_width': 'initial'}
)

# Display the widget
display(model_dropdown)

Dropdown(description='Model:', options=('Llama 3', 'Mistral', 'SmolLM'), style=DescriptionStyle(description_wi…

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Function to load the selected model
def load_model(model_name):
    model_map = {
        "Llama 3": "meta-llama/Llama-3.1-8B",
        "Mistral": "mistralai/Mistral-7B-v0.1",
        "SmolLM": "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
    }

    if model_name in model_map:
        model_id = model_map[model_name]
        print(f"Loading {model_name} ({model_id})...")

        tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=True)
        model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", use_auth_token=True)

        print(f"{model_name} loaded successfully!")
        return model, tokenizer
    else:
        print("Invalid model selection.")
        return None, None

# Load the model based on user selection
selected_model = model_dropdown.value
model, tokenizer = load_model(selected_model)

Loading SmolLM (TinyLlama/TinyLlama-1.1B-Chat-v1.0)...


/opt/conda/lib/python3.11/site-packages/transformers/models/auto/tokenization_auto.py:862: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

/opt/conda/lib/python3.11/site-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

SmolLM loaded successfully!


In [5]:
import pandas as pd
import json
import io
import ipywidgets as widgets
from IPython.display import display

# Create file uploader widget
file_uploader = widgets.FileUpload(accept=".json,.csv,.jsonl", multiple=False)
display(file_uploader)

# Function to read and validate dataset
def process_dataset(uploaded_files):
    if not uploaded_files:
        print("❌ No file uploaded.")
        return None

    # Extract the first uploaded file from tuple
    uploaded_file = uploaded_files[0]  # Fix: Extract from tuple

    # Access name and content
    file_name = uploaded_file.name
    content = uploaded_file.content  # Extract file content as bytes

    # Convert content to BytesIO for pandas
    file_stream = io.BytesIO(content)

    # Read file based on format
    if file_name.endswith(".csv"):
        df = pd.read_csv(file_stream)
    elif file_name.endswith(".json"):
        df = pd.read_json(file_stream)
    elif file_name.endswith(".jsonl"):
        content_str = content.decode("utf-8").split("\n")
        data = [json.loads(line) for line in content_str if line.strip()]
        df = pd.DataFrame(data)
    else:
        print("❌ Unsupported file format.")
        return None

    print(f"✅ Dataset Loaded: {file_name}")
    print(df.head())  # Show first few rows
    return df

# Function to handle file upload event
def on_upload(change):
    global dataset
    dataset = process_dataset(file_uploader.value)  # Process dataset

# Attach the event listener
file_uploader.observe(on_upload, names="value")

FileUpload(value=(), accept='.json,.csv,.jsonl', description='Upload')

In [6]:
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

def prepare_hf_dataset(df, text_column="text", extract_key=None, max_length=512):
    global tokenizer  

    if text_column not in df.columns:
        raise ValueError(f"❌ Column '{text_column}' not found in dataset!")

    # Extract the key from the dictionary if needed
    if extract_key:
        df[text_column] = df[text_column].apply(lambda x: x.get(extract_key, []) if isinstance(x, dict) else [])

    # Flatten the list of patterns into individual rows
    df = df.explode(text_column)

    # Split into train, validation, and test (80-10-10)
    train_texts, temp_texts = train_test_split(df[text_column].dropna().tolist(), test_size=0.2, random_state=42)
    val_texts, test_texts = train_test_split(temp_texts, test_size=0.5, random_state=42)

    # Tokenization
    tokenized_dataset = DatasetDict({
        "train": Dataset.from_dict(tokenizer(train_texts, truncation=True, padding="max_length", max_length=max_length)),
        "validation": Dataset.from_dict(tokenizer(val_texts, truncation=True, padding="max_length", max_length=max_length)),
        "test": Dataset.from_dict(tokenizer(test_texts, truncation=True, padding="max_length", max_length=max_length)),
    })

    print("✅ Dataset prepared successfully!")
    return tokenized_dataset
    
hf_dataset = prepare_hf_dataset(dataset, text_column="intents", extract_key="patterns")

# Show dataset structure
print(hf_dataset)

✅ Dataset prepared successfully!
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 324
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 40
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 41
    })
})


In [15]:
import torch
import optuna
import time
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import DatasetDict
import evaluate  # ✅ Replaces deprecated load_metric
import numpy as np

# Ensure the dataset is correctly assigned
hf_dataset = DatasetDict({
    "train": hf_dataset["train"],
    "validation": hf_dataset["validation"],
    "test": hf_dataset["test"]
})

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # No masked language modeling
)

# Ensure dataset items are tensors
def format_dataset(example):
    return {
        "input_ids": torch.tensor(example["input_ids"], dtype=torch.long),
        "attention_mask": torch.tensor(example["attention_mask"], dtype=torch.long),
        "labels": torch.tensor(example["input_ids"], dtype=torch.long),
    }

hf_dataset = hf_dataset.map(format_dataset)

# ✅ Replace deprecated load_metric with evaluate.load
perplexity_metric = evaluate.load("perplexity")
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")
accuracy_metric = evaluate.load("accuracy")

# Interactive UI for hyperparameter selection
learning_rate_slider = widgets.FloatLogSlider(
    value=5e-5, base=10, min=-6, max=-3, step=0.1,
    description="Learning Rate", style={'description_width': 'initial'}
)
batch_size_slider = widgets.IntSlider(
    value=8, min=2, max=64, step=2,
    description="Batch Size", style={'description_width': 'initial'}
)
epochs_slider = widgets.IntSlider(
    value=3, min=1, max=10, step=1,
    description="Epochs", style={'description_width': 'initial'}
)
gradient_accumulation_slider = widgets.IntSlider(
    value=4, min=1, max=16, step=1,
    description="Gradient Accumulation", style={'description_width': 'initial'}
)
weight_decay_slider = widgets.FloatLogSlider(
    value=0.01, base=10, min=-5, max=0, step=0.1,
    description="Weight Decay", style={'description_width': 'initial'}
)

# Display widgets
display(learning_rate_slider, batch_size_slider, epochs_slider, gradient_accumulation_slider, weight_decay_slider)

# Function to define training arguments
def get_training_args(
    learning_rate=5e-5,
    batch_size=8,
    epochs=3,
    gradient_accumulation_steps=1,
    weight_decay=0.01
):
    return TrainingArguments(
        output_dir="./results",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        gradient_accumulation_steps=gradient_accumulation_steps,
        eval_strategy="epoch",
        save_strategy="epoch",
        logging_dir="./logs",
        logging_steps=10,
        load_best_model_at_end=True,
        report_to="tensorboard",
        remove_unused_columns=False,
        disable_tqdm=True  # ✅ Disable progress bars
    )

# Optuna hyperparameter tuning
def objective(trial):
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 1e-3)
    batch_size = trial.suggest_categorical("batch_size", [4, 8, 16, 32])
    epochs = trial.suggest_int("epochs", 2, 6)
    grad_accum = trial.suggest_int("gradient_accumulation", 1, 8)
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-5, 1e-1)

    training_args = get_training_args(learning_rate, batch_size, epochs, grad_accum, weight_decay)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=hf_dataset["train"],
        eval_dataset=hf_dataset["validation"],
        tokenizer=tokenizer,
        data_collator=data_collator
    )

    trainer.train()
    eval_results = trainer.evaluate()
    return eval_results["eval_loss"]

# Automatically run Optuna tuning if user does not modify defaults
use_optuna = True  # Assume tuning unless user changes defaults
default_values = {
    "learning_rate": 5e-5,
    "batch_size": 8,
    "epochs": 3,
    "gradient_accumulation": 4,
    "weight_decay": 0.01
}

def check_user_changes():
    global use_optuna
    use_optuna = not (
        learning_rate_slider.value != default_values["learning_rate"] or
        batch_size_slider.value != default_values["batch_size"] or
        epochs_slider.value != default_values["epochs"] or
        gradient_accumulation_slider.value != default_values["gradient_accumulation"] or
        weight_decay_slider.value != default_values["weight_decay"]
    )

# Button to start training
train_button = widgets.Button(description="Start Training", button_style="success")
output = widgets.Output()

def on_train_click(b):
    global use_optuna
    with output:
        clear_output(wait=True)
        check_user_changes()

        if use_optuna:
            print("🔍 Running Hyperparameter Tuning with Optuna...")
            study = optuna.create_study(direction="minimize")
            study.optimize(objective, n_trials=10)
            best_params = study.best_params
            print(f"🎯 Best Hyperparameters Found: {best_params}")
        else:
            best_params = {
                "learning_rate": learning_rate_slider.value,
                "batch_size": batch_size_slider.value,
                "epochs": epochs_slider.value,
                "gradient_accumulation_steps": gradient_accumulation_slider.value,
                "weight_decay": weight_decay_slider.value
            }

        print("🚀 Starting Training with Selected Parameters...")
        training_args = get_training_args(**best_params)

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=hf_dataset["train"],
            eval_dataset=hf_dataset["validation"],
            tokenizer=tokenizer,
            data_collator=data_collator
        )

        trainer.train()
        print("✅ Training Completed!")

        print("📊 Evaluating Model...")
        eval_results = trainer.evaluate()
        for key, value in eval_results.items():
            print(f"{key}: {value:.4f}")

        print("🎯 Training & Evaluation Complete!")

train_button.on_click(on_train_click)
display(train_button, output)


Map:   0%|          | 0/324 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/41 [00:00<?, ? examples/s]

FloatLogSlider(value=5e-05, description='Learning Rate', max=-3.0, min=-6.0, style=SliderStyle(description_wid…

IntSlider(value=8, description='Batch Size', max=64, min=2, step=2, style=SliderStyle(description_width='initi…

IntSlider(value=3, description='Epochs', max=10, min=1, style=SliderStyle(description_width='initial'))

IntSlider(value=4, description='Gradient Accumulation', max=16, min=1, style=SliderStyle(description_width='in…

FloatLogSlider(value=0.01, description='Weight Decay', max=0.0, min=-5.0, style=SliderStyle(description_width=…

Button(button_style='success', description='Start Training', style=ButtonStyle())

Output()

In [13]:
pip install evaluate rouge_score nltk


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 62.8 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=61b4f53f30267cdcb105388aa3afd96f7b58d5f803d6980dc3c20051c4a689b6
  Stored in directory: /home/jovyan/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.
